# Integrate KF into class

In [1]:
import filterpy
from filterpy.common import Q_discrete_white_noise
import numpy as np
import h5py
import pandas as pd
from filterpy.kalman import KalmanFilter
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import scipy
from scipy.spatial.distance import cdist

In [2]:
# upload data 
hdf5data_109 = h5py.File('../../Data/data_109.h5', 'r')
hdf5data_130 = h5py.File('../../Data/data_130.h5', 'r')
hdf5data_142 = h5py.File('../../Data/data_142.h5', 'r')
hdf5data_143 = h5py.File('../../Data/data_143.h5', 'r')

In [3]:
def fill_rows_detections(detections, returndf):
    
    coord_detections = [np.array(vals[0].tolist()) for vals in detections]
    coord_detections = np.vstack(coord_detections)
        
    length_box = [vals[1] for vals in detections]
    width_box = [vals[2] for vals in detections]
    height_box = [vals[3] for vals in detections]
    angle_box = [vals[4] for vals in detections]
    
    returndf["X_box"] = coord_detections[:,0]
    returndf["Y_box"] = coord_detections[:,1]
    returndf["Z_box"] = coord_detections[:,2]
    returndf["length_box"], returndf["width_box"], returndf["height_box"] = length_box, width_box, height_box
    returndf["angle_box"] = angle_box
    
    return returndf

In [4]:
def df_detections(h5data,camera = None):
    
    timestamps = h5data['Timestamp']
    df = pd.DataFrame()
    
    for c, t in enumerate(timestamps):
        
        append_df = pd.DataFrame()
        
        d = h5data['Sequence'][str(c)]
        detection = np.asarray(d['Detections'])
        
        if detection.size:
            append_df = fill_rows_detections(detection, append_df)
            append_df['timestamp'] = t
            append_df['frame'] = c
        
            if camera is not None:
                append_df['camera'] = camera
        
            df = df.append(append_df)
    
    return df

In [5]:
df_109 = df_detections(hdf5data_109, 109)
df_109.reset_index(inplace=True, drop=True)
df_130 = df_detections(hdf5data_130, 130)
df_130.reset_index(inplace=True, drop=True)
df_142 = df_detections(hdf5data_142, 142)
df_142.reset_index(inplace=True, drop=True)
df_143 = df_detections(hdf5data_143, 143)
df_143.reset_index(inplace=True, drop=True)

In [6]:
def concatenate_dfs(dfs, sort_cols):
    
    df_to_concat = dfs
    df_complete = pd.concat(df_to_concat)
    df_complete.sort_values(by = sort_cols, inplace = True)
    df_complete.reset_index(inplace=True, drop=True)
    df_complete['elapsed_time'] = df_complete['timestamp']-df_complete['timestamp'].min() # my addition, can be useful
    
    return df_complete

df_complete = concatenate_dfs([df_109,df_130,df_142,df_143] , ['frame', 'camera'])
df_complete

,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
0,-11.148409,15.363879,-0.941920,1.177285,0.561967,1.334849,0.631927,1.574333e+09,0,109,0.000000
1,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,0,109,0.000000
2,3.373729,28.175323,-1.134802,5.493111,1.930805,2.056836,1.514860,1.574333e+09,0,109,0.000000
3,6.546980,26.314280,-0.862299,1.506848,0.592773,1.134711,0.424306,1.574333e+09,0,109,0.000000
4,14.382445,-4.612707,-1.015361,4.811726,2.805154,1.550568,1.004918,1.574333e+09,0,130,0.047641
...,...,...,...,...,...,...,...,...,...,...,...
11740,-5.128264,26.316446,-1.150593,2.670343,1.157835,1.589266,-0.424405,1.574333e+09,749,109,59.920729
11741,-1.906789,-5.313489,-2.293509,3.833054,0.585950,0.395508,-0.462055,1.574333e+09,749,142,59.927601
11742,-1.142268,-3.922715,-2.524621,12.753714,0.377229,1.433762,-0.278929,1.574333e+09,749,142,59.927601
11743,14.617583,-29.436074,-1.085488,5.046473,2.802799,1.898150,-1.309933,1.574333e+09,749,143,59.967542


In [7]:


#implementing constant acceleration model. Matrix F can be recycled as it does not change
f = KalmanFilter (dim_x=9, dim_z=3)
f.x = np.array([0.,0.,0.,0.5,0.5,0.5,0.1,0.1,0.1])
dt = 0.01
f.F = np.array([[1., dt, 0.5*dt**2, 0., 0., 0., 0., 0., 0.], [0, 1, dt, 0., 0., 0., 0., 0., 0. ], [0.,0.,1., 0., 0., 0., 0., 0., 0.],
               [0.,0.,0.,1.,dt,0.5*dt**2,0.,0.,0.],[0.,0.,0.,0.,1.,dt,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.],
               [0.,0.,0.,0.,0.,0.,1.,dt,0.5*dt**2],[0.,0.,0.,0.,0.,0.,0.,1.,dt], [0.,0.,0.,0.,0.,0.,0.,0.,1.]])
f.H = np.array([[1.,0.,0., 0., 0., 0., 0., 0., 0.],[0.,0., 0.,1., 0., 0., 0., 0., 0.],[0.,0., 0., 0., 0., 0.,1., 0., 0.]])
f.P *= 0.1
f.Q = np.eye(9)*0.1
f.R = np.eye(3)

In [8]:
def linear_transition_matrix(dt):
    return np.array([[1., dt, 0.5*dt**2, 0., 0., 0., 0., 0., 0.], [0, 1, dt, 0., 0., 0., 0., 0., 0. ], [0.,0.,1., 0., 0., 0., 0., 0., 0.],
               [0.,0.,0.,1.,dt,0.5*dt**2,0.,0.,0.],[0.,0.,0.,0.,1.,dt,0.,0.,0.],[0.,0.,0.,0.,0.,1.,0.,0.,0.],
               [0.,0.,0.,0.,0.,0.,1.,dt,0.5*dt**2],[0.,0.,0.,0.,0.,0.,0.,1.,dt], [0.,0.,0.,0.,0.,0.,0.,0.,1.]])

In [62]:
deltaT = dt= 0.08
threshold = 7
ACTIVE_TIME_THRESHOLD = 8
SPEED_ACC_WINDOW = 5

class Track:
    
    def __init__(self, ID, OBJ, FRAME, threshold = threshold):
        self.ID = ID
        self.status = 'New'  #New, Pending, Active, Inactive
        self.objects = [OBJ]
        self.x, self.y, self.z = OBJ.x, OBJ.y, OBJ.z
        self.frames = [FRAME]
        self.velocity = [0.5, 0.5, 0.5]
        self.acceleration = [0.2, 0.2, 0.2]
        self.type = None
        self.threshold = threshold
        self.filter = None
        self.set_Kalman_Filter()
        self.new_time = 1
        self.pending_time = 0
        self.active_time = 0
    
    def update(self, OBJ, FRAME):
        
        if OBJ is not None:
            self.objects.append(OBJ)
            self.frames.append(FRAME)
            if self.status == 'New':
                #run iou tracker to get the estimate of speed, and update KF accordingly. 
                # In the meantime, get the next position from the object
                # assuming obj contains the right thing, here we adopt a different approach on it dep on the status
                self.x, self.y, self.z = OBJ.x, OBJ.y, OBJ.z
                last_xs = np.array([obj.x for obj in self.objects])
                last_ys = np.array([obj.y for obj in self.objects])
                last_zs = np.array([obj.z for obj in self.objects])
                if len(last_xs)>1:
                    self.velocity[0] = np.mean(last_xs[1:]-last_xs[:-1]/deltaT)
                    self.velocity[1] = np.mean(last_ys[1:]-last_ys[:-1]/deltaT)
                    self.velocity[2] = np.mean(last_zs[1:]-last_zs[:-1]/deltaT)
                if len(last_xs)>2:
                    self.acceleration[0] = (last_xs[2:] - 2* last_xs[1:-1] + last_xs[:-2])[0]/(deltaT**2)
                    self.acceleration[1] = (last_ys[2:] - 2* last_ys[1:-1] + last_ys[:-2])[0]/(deltaT**2)
                    self.acceleration[2] = (last_zs[2:] - 2* last_zs[1:-1] + last_zs[:-2])[0]/(deltaT**2)
                self.filter.x = np.array([self.x,self.velocity[0], self.acceleration[0],
                                          self.y,self.velocity[1], self.acceleration[1],
                                          self.z,self.velocity[2], self.acceleration[2]])
                self.new_time += 1
                if self.new_time > self.threshold:
                    self.status = 'Active'
            else:
                # how do I get the elapsed time between observations? assume we put it in dt
                self.filter.F = linear_transition_matrix(dt)
                # self.x, self.y, self.z = OBJ.x, OBJ.y, OBJ.z #We should use KF.filter
                self.update_Kalman_Filter(dt, np.array([OBJ.x, OBJ.y, OBJ.z]))

                if self.active_time < ACTIVE_TIME_THRESHOLD: # keep using iou while also training KF
                    self.x, self.y, self.z = OBJ.x, OBJ.y, OBJ.z
                    last_xs = np.array([obj.x for obj in self.objects[-SPEED_ACC_WINDOW:]])
                    last_ys = np.array([obj.y for obj in self.objects[-SPEED_ACC_WINDOW:]])
                    last_zs = np.array([obj.z for obj in self.objects[-SPEED_ACC_WINDOW:]])
                    
                    self.velocity[0] = np.mean(last_xs[1:]-last_xs[:-1]/deltaT)
                    self.velocity[1] = np.mean(last_ys[1:]-last_ys[:-1]/deltaT)
                    self.velocity[2] = np.mean(last_zs[1:]-last_zs[:-1]/deltaT)
                    
                    self.acceleration[0] = (last_xs[2:] - 2* last_xs[1:-1] + last_xs[:-2])[0]/(deltaT**2)
                    self.acceleration[1] = (last_ys[2:] - 2* last_ys[1:-1] + last_ys[:-2])[0]/(deltaT**2)
                    self.acceleration[2] = (last_zs[2:] - 2* last_zs[1:-1] + last_zs[:-2])[0]/(deltaT**2)
                    
                    
                else:
                    self.x = self.filter.x[0] # maybe these are not needed as we have the filter ones
                    self.y = self.filter.x[3]
                    self.z = self.filter.x[6]
                    self.velocity = [self.filter.x[1], self.filter.x[4], self.filter.x[7]]
                    self.acceleration = [self.filter.x[2], self.filter.x[5], self.filter.x[8]]
                self.status = 'Active'
            
            # adding: if track is new look for previous frames and get iou estimate of the speed 
        
        if OBJ is None and self.status == 'New':
            if self.new_time == 1: self.status = 'Removed'
            else: self.new_time += 1
        
        if OBJ is None and self.status == 'Active':
            self.status = 'Pending'
            self.pending_time += 1
        
        if OBJ is None and self.status == 'Pending':
            if self.pending_time == 5: self.status = 'Inactive'
            else: self.pending_time += 1
        
        return
    
    def set_status(self, status):
        self.status = status
        
        return
    
    
    
#     def set_type( self, TYPE ):
#         ### NOT USED ###
#         self.type = TYPE
#         set_threshold()
#         return
    
#     def set_threshold (self):
#         ### NOT USED ###
#         if type == 'car':
#             self.threshold = 3
        
#         else:
#             self.threshold = 10
            
#         return
    
    def to_df(self):
        '''Print Trajectory as Dataframe'''
        df = pd.DataFrame(columns = ['X_box', 'Y_box', 'Z_box', 'length_box', 'width_box', 'height_box', 'angle_box', 'camera'])
        for i, j in enumerate(self.objects):
            append_df = j.to_df()
            append_df['frame'] = self.frames[i]       
            df = df.append(append_df)
            df.reset_index(inplace=True, drop=True)
            
        return df
    
    def set_Kalman_Filter(self, model_type='ca', x0=None, dt=0.1):
        if x0 is None:
            x0 = np.array([0, 0.5, 0.2, 0, 0.5, 0.2, 0, 0.5, 0.2])
        if model_type == 'ca':
            self.filter = KalmanFilter (dim_x=9, dim_z=3)
            self.filter.x = x0
            self.filter.F = linear_transition_matrix(dt)
            self.filter.H = np.array([[1.,0.,0., 0., 0., 0., 0., 0., 0.],[0.,0., 0.,1., 0., 0., 0., 0., 0.],[0.,0., 0., 0., 0., 0.,1., 0., 0.]])
            self.filter.P *= 0.1 # TO ADJUST
            self.filter.Q = np.eye(9)*0.005 # TO ADJUST
            self.filter.R = np.diag(np.array([0.5, 0.2, 0.05]))*0.01 # TO ADJUST
            
    def update_Kalman_Filter(self, dt, y):
        y = np.array(y)
        self.filter.predict()
        self.filter.update(y)
        return
    
    def __str__(self):
        return "ID: %d, status: %s, x: %f, y:%f, z:%f, new_time: %f, pending_time: %f, objects_count: %d, frame_count: %d"%(self.ID, self.status, self.x, self.y, self.z, self.new_time, self.pending_time, len(self.objects), len(self.frames))
    def __repr__(self):
        return "ID %d - TimeNew %d\n - Last coords (%f,%f,%f)" %(self.ID, self.new_time, self.x, self.y, self.z)
    

class Object:
    def __init__(self, x, y, z, l, w, h, angle, camera):
        self.x = x
        self.y = y
        self.z = z
        self.length = l
        self.width = w
        self.height = h
        self.angle = angle
        self.camera = camera
        
    def to_df(self):
        
        df = pd.DataFrame([[self.x, self.y, self.z, self.length, self.width, self.height, self.angle, self.camera]], 
                         columns = ['X_box', 'Y_box', 'Z_box', 'length_box', 'width_box', 'height_box', 'angle_box', 'camera'])

        return df
    
    def __repr__(self):
        return "x: %f, y:%f, z:%f, length:%f, width:%f, height:%f, angle:%f, camera:%d \n" %(self.x, self.y, self.z, self.length, self.width, self.height, self.angle, self.camera)
    
    def __str__(self):
        return "x: %f, y:%f, z:%f, length:%f, width:%f, height:%f, angle:%f, camera:%d \n" %(self.x, self.y, self.z, self.length, self.width, self.height, self.angle, self.camera)

In [64]:
#Fill this functions with chosen approach
MAX_DIST = 1

#Using Nearest Neighbour -> not right, will always connect a track to an object
def track_detection_association(track_coord, obj_coord):
    ''' Function for association between detections and tracks'''
    
    dists = cdist(track_coord, obj_coord, 'euclidean') 
    associations = np.argmin(dists, axis=1)
    
    associations[np.min(dists, axis = 1) > MAX_DIST] = -1
    #print(associations)
    return associations, np.min(dists, axis = 1)

# def associate(track_coord, tracks, status, obj_list, frame_index): # can we instead pass all the track, so that I can check also iou?
def associate(tracks, obj_list, frame_index):
    
    track_coord = np.array([[t.x, t.y, t.z] for t in selected_tracks]) 
    
    if track_coord.size and len(obj_list) > 0:
        
        
        # explore with iou
        #associations = find_intersecting_boxes_df(tracks_to_link_df, pd.DataFrame(obj_list), threshold=0) #ideally this returns alternative associations
        #associations = find_subsequent_boxes(selected_tracks, obj_list)
        associations, iou = find_subsequent_boxes(tracks, obj_list, return_iou=True)
        print("Iou associations")
        print(iou)
        print(associations)


        associations, iou = find_subsequent_boxes(selected_tracks, obj_list, return_iou=True)
        obj_coord = [(obj.x, obj.y, obj.z) for obj in obj_list]
        KF_associations, dists = track_detection_association(track_coord, obj_coord)
        indices_KF = [i for i in range(len(selected_tracks)) if selected_tracks[i].active_time >= ACTIVE_TIME_THRESHOLD ]
        # problem: there can be multiple associations this way
        associations[indices_KF] = KF_associations[indices_KF]
        print("KF associations")
        print(dists)
        print(associations)
        
        for index, ass in enumerate(associations):
            #print(ass)
            if ass == -1:
                tracks[status][index].update(None, frame_index)
            else:
                tracks[status][index].update(obj_list[ass], frame_index)

        obj_list = np.delete(obj_list, associations)
    
    return tracks, obj_list

In [16]:
def belongs_to(point, box, isdf=False):
    #print(point)
    if isdf:
        center = box[['X_box', 'Y_box', 'Z_box']].to_numpy().flatten()
        point = np.array([i for i in point]) - center
        anti_rotation_matrix = np.array([[np.cos(-box['angle_box'].to_numpy()[0]), -np.sin(-box['angle_box'].to_numpy()[0]), 0], 
                            [np.sin(-box['angle_box'].to_numpy()[0]), np.cos(-box['angle_box'].to_numpy()[0]), 0], 
                            [0,0,1]])
        rot_point = anti_rotation_matrix.dot(point)
        #print("point: ", point)
        #print(rot_point)
        if np.abs(rot_point[0])<box['length_box'].to_numpy()/2 and np.abs(rot_point[1])<box['width_box'].to_numpy()/2 and np.abs(rot_point[2])<box['height_box'].to_numpy()/2:
            return True
        return False
    
    center = np.array([i for i in box[0]])
    point = np.array([i for i in point]) - center
    anti_rotation_matrix = np.array([[np.cos(-box[4]), -np.sin(-box[4]), 0], 
                            [np.sin(-box[4]), np.cos(-box[4]), 0], 
                            [0,0,1]])
    rot_point = anti_rotation_matrix.dot(point)
    
    if np.abs(rot_point[0])<box[1]/2 and np.abs(rot_point[1])<box[2]/2 and np.abs(rot_point[2])<box[3]/2:
        return True
    return False

def compute_approx_IoU(box1, box2, attempts=100): #TODO increment number iterations
    
    #print(box1)
    #print(box2)
    

    pos1 = box1[['X_box', 'Y_box', 'Z_box']].to_numpy().flatten()
    pos2 = box2[['X_box', 'Y_box', 'Z_box']].to_numpy().flatten()
    #print(pos1, pos2)
    length1 = box1['length_box'].to_numpy()[0]
    width1 = box1['width_box'].to_numpy()[0]
    height1 = box1['height_box'].to_numpy()[0]
    length2 = box2['length_box'].to_numpy()[0]
    width2 = box2['width_box'].to_numpy()[0]
    height2 = box2['height_box'].to_numpy()[0]
    diag1 = np.sqrt(length1**2+width1**2+height1**2)
    diag2 = np.sqrt(length2**2+width2**2+height2**2)
    rotation_matrix1 = np.array([[np.cos(box1['angle_box'].to_numpy())[0], -np.sin(box1['angle_box'].to_numpy()[0]), 0], 
                            [np.sin(box1['angle_box'].to_numpy()[0]), np.cos(box1['angle_box'].to_numpy()[0]), 0], 
                            [0,0,1]])
    rotation_matrix2 = np.array([[np.cos(box2['angle_box'].to_numpy()[0]), -np.sin(box2['angle_box'].to_numpy()[0]), 0], 
                            [np.sin(box2['angle_box'].to_numpy()[0]), np.cos(box2['angle_box'].to_numpy()[0]), 0], 
                            [0,0,1]])
    
    
    if np.sqrt((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2+(pos1[2]-pos2[2])**2)>max(diag1, diag2):
        return 0
    intersection = 0
    #print("arr",np.array([length1,width1,height1]))
    for i in range(attempts):
        u = np.random.uniform(-1,1,size=3) * np.array([length1,width1,height1])/2 
        #print(np.array([i for i in pos1]))
        #print("u: ", u)
        u = rotation_matrix1.dot(u)
        if belongs_to(u + np.array([i for i in pos1]), box2, True):
            intersection +=1
        u = np.random.uniform(-1,1,size=3) * np.array([length2,width2,height2])/2 
        u = rotation_matrix2.dot(u)
        if belongs_to(u + np.array([i for i in pos2]), box1, True):
            intersection +=1
    return intersection/2/attempts

# def find_intersecting_boxes_df(detection1, detection2, threshold=0.2, all_matches=False):
#     return_df = pd.DataFrame(columns=['camera1', 'camera2','original_idx1','original_idx2','iou', 'bestmatch1', 'bestmatch2'])
    
#     for idx, det in detection1.iterrows():
#         for idxx, dett in detection2.iterrows():
    
#             iou = compute_approx_IoU(det,dett)
#             if iou > threshold:
#                 return_df = return_df.append({'camera1': det['camera'], 'camera2': dett['camera'],'original_idx1':idx, 'original_idx2':idxx, 'iou': iou, 'bestmatch1': False, 'bestmatch2': False}, ignore_index=True)
    
#     best1idx = return_df.groupby(['camera1', 'original_idx1'])['iou'].idxmin()            
#     best2idx = return_df.groupby(['camera2', 'original_idx2'])['iou'].idxmin() 
#     return_df.loc[best1idx, 'bestmatch1'] = True
#     return_df.loc[best2idx, 'bestmatch2'] = True
#     if all_matches:
#         return return_df
#     return return_df.loc[return_df[['bestmatch1', 'bestmatch2']].any()]

def find_subsequent_boxes(track_list1, track_list2, threshold=0.2, return_iou=False):
    """
    Returns a numpy array of the best matches of boxes in track_list2 for boxes in track_list1, if any 
    If no IoU above a fixed threshold is found, -1 is assigned.
    Optionally it will also return ious
    """
    
    iou_vec = np.zeros(shape=len(track_list1))
    matches = -np.ones(shape=len(track_list1), dtype=np.int8)
    for i in range(len(track_list1)):
        for j in range(len(track_list2)):
#             print(track_list1[i])
#             print(track_list2[j])
#             print(track_list1[['X_box', 'Y_box', 'Z_box']].to_numpy().flatten())
            df1 = track_list1[i].to_df()
            #df2 = track_list2[j].to_df()
            last_frame = df1.frame.max()
            iou = compute_approx_IoU(df1[df1.frame==last_frame],track_list2[j].to_df())
            if iou > iou_vec[i] and iou > threshold: # can be refined to also account for very different volumes
                iou_vec[i] = iou
                matches[i] = j
        #print(matches)        
        #can use this to check if there is any split, uncertainty, etc
        if not return_iou:
            return matches
    return matches, iou_vec

In [13]:
clock_rate = 0.08

class Tracker:
    
    def __init__(self, dfs):
        self.tracks = {'Active': [], 'Pending': [], 'Inactive': [], 'New': [], 'Removed': []}
        self.dataframe = concatenate_dfs(dfs, ['frame', 'camera'])
        self.track_id = 0 # is this a counter?
        self.time = -0.06
        self.current_objects = pd.DataFrame()#self.dataframe[np.abs(self.dataframe['elapsed_time']-self.time)<=clock_rate/2] # storing all detections of the latest-observed time step, namely the one we are analyzing
        self.frame_index = -1
        
        # should we also call self.analyze_frame here to initialize the tracks?
        # self.frame_index = 0 # problem of asynchro. We might solve it by having an internal clock moving say constantly 
        # it has a pb tho, and it is that if 2 frames are closer than 0.08 they might appear on the same, so need to check the camera and give a criterium for deciding
        # at rate 0.08 or we might choose a reference camera for it. Maybe the camera with best visual. This is a second step though, 
        # because first you must be confident in detecting which camera is central for an object
    
    def update_tracks(self):
        ''' Update the status of the tracks, moving them to the different lists'''
        for tr in self.tracks['New'][::-1]:
            if tr.status != 'New':
                self.tracks['New'].remove(tr)
                self.tracks[tr.status].append(tr)
        
        for tr in self.tracks['Pending'][::-1]:
            if tr.status != 'Pending':
                self.tracks['Pending'].remove(tr)
                self.tracks[tr.status].append(tr)
        
        for tr in self.tracks['Active'][::-1]:
            if tr.status != 'Active':
                self.tracks['Active'].remove(tr)
                self.tracks[tr.status].append(tr)
        
        return
    
    def next_step(self):
        '''Proceed with next time step analysis of frames'''
        #''' Increase time step of the camera'''
        self.frame_index += 1
        self.time += clock_rate
        self.current_objects = self.dataframe[np.abs(self.dataframe['elapsed_time']-self.time)<=clock_rate/2]# very unlikely one happens to be EXACTLY in the middle
        #self.merge_collisions()
        self.analyze_frame()
    
#     def explore_with_iou(self, category='New'):
#         '''Run an exploratory IoU search in order to link objects independently of Kalman Filter. 
#         It may implement different logic for different track categories'''
#         if category == 'New':
#             exploration = find_intersecting_boxes_df()
        
    
    def analyze_frame(self):
        ''' Analyze frame creating or updating trajectories'''
        
#         frame = self.dataframe[self.dataframe['frame'] == self.frame_index]
#         frame.reset_index(inplace=True, drop=True)
        
        frame = self.current_objects 
        obj_list = []
        
        
        # Extract all the detections in the frame
        for index, row in frame.iterrows():
            obj = Object(row['X_box'], row['Y_box'], row['Z_box'], row['length_box'], 
                         row['width_box'], row['height_box'], row['angle_box'], row['camera'])
            obj_list.append(obj)

        obj_list = np.array(obj_list)
        
        # At first we use Active Tracks
        #track_act_coord = np.array([[t.x, t.y, t.z] for t in self.tracks['Active']]) #We should use KF.predict here and not consider only the new set
        #self.tracks, obj_list = associate(track_act_coord, self.tracks, 'Active', obj_list, self.frame_index)
        self.tracks, obj_list = associate( self.tracks, 'Active', obj_list, self.frame_index)

        #Then we use tracks set as New
        #track_new_coord = np.array([[t.x, t.y, t.z] for t in self.tracks['New']]) #We should use KF.predict here and not consider only the new set
        self.tracks, obj_list = associate(self.tracks, 'New', obj_list, self.frame_index)

        #Then we use tracks set as Pending
        #track_pending_coord = np.array([[t.x, t.y, t.z] for t in self.tracks['Pending']]) #We should use KF.predict here and not consider only the new set
        self.tracks, obj_list = associate(self.tracks, 'Pending', obj_list, self.frame_index)

        
        # Check status of all tracks
        self.update_tracks()
        # frame should also include information about the status
        
        
        #Create new tracks for not linked objects
        for obj in obj_list:
            tr = Track(self.track_id, obj, self.frame_index)
            self.track_id += 1
            self.tracks['New'].append(tr)
        
        #Go to the next frame - delegated to next step 
        # self.next_step()
        
    def merge_collisions(self):
        
        # version that only considers the situation of same camera observed twice in a timeframe
#         camera_time_counts = self.current_objects.groupby(['camera']).elapsed_time.nunique()
        
#         if camera_time_counts.min()==1:
#             return
        
#         new_df = pd.DataFrame()
#         for camera in camera_time_counts.index:
#             # some function adding one column to the df (temporarily) of name "identified_obj_idx"
#             temporary_df = function(self.current_objects.loc[self.current_objects['camera']==camera])
#             new_df = new_df.append(temporary_df.groupby(['identified_obj_idx']).mean(), ignore_index=True)
#         self.current_objects = new_df   

        # extension to the general case in which we could label all unique objects even from different cameras, inlcuding weights
    
        #required format: columns of current_objects + 'object_index' + 'weight'
        #labeled_df = find_object_grouping(self.current_objects)
        labeled_df = None
        if np.all(labeled_df.groupby(['labels']).count()==1):
            return 
        
        labeled_weighted_df = labeled_df.drop('weight', axis=1)
        labeled_weighted_df.loc[:,labeled_df.columns not in ['weight', 'camera', 'object_index']] = labeled_df.loc[:,labeled_df.columns not in ['weight', 'camera', 'object_index']].multiply(labeled_df.weights, axis='index')
        
        self.current_objects = labeled_weighted_df.groupby(['object_index']).mean()
        self.current_objects['elapsed_time'] = self.time
        
        
        return

In [ ]:
df_complete

In [134]:
TrackTrack = Tracker([df_109,df_130,df_142,df_143])

In [135]:
TrackTrack.tracks

{'Active': [], 'Pending': [], 'Inactive': [], 'New': [], 'Removed': []}

In [106]:
TrackTrack.time

-0.06

In [ ]:
t = Track(1,Object(1,2,3,4,5,6,7,109), 1)

In [ ]:
t.to_df()

In [132]:
TrackTrack.next_step()

[0.755 0.505 0.73  0.825 0.78  0.    0.95  0.69  0.    0.425 0.    0.78
 0.845 0.955]
[ 3  0  1  2  4 -1  5  7 -1  8 -1  9 10 11]


In [94]:
TrackTrack.current_objects

,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
0,-11.148409,15.363879,-0.941920,1.177285,0.561967,1.334849,0.631927,1.574333e+09,0,109,0.000000
1,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,0,109,0.000000
2,3.373729,28.175323,-1.134802,5.493111,1.930805,2.056836,1.514860,1.574333e+09,0,109,0.000000
3,6.546980,26.314280,-0.862299,1.506848,0.592773,1.134711,0.424306,1.574333e+09,0,109,0.000000
4,14.382445,-4.612707,-1.015361,4.811726,2.805154,1.550568,1.004918,1.574333e+09,0,130,0.047641
5,27.471277,9.154471,-0.657809,2.640507,0.653304,1.245023,-0.802791,1.574333e+09,0,130,0.047641
6,10.591985,-33.300211,-1.254730,3.106008,2.026976,2.356834,1.390347,1.574333e+09,0,143,0.047386
7,2.417817,-32.064833,-0.609711,0.641007,0.489730,0.741013,-0.615712,1.574333e+09,0,143,0.047386
8,13.233552,-5.117304,-0.707404,9.933850,2.673657,1.161759,1.067465,1.574333e+09,0,143,0.047386


In [72]:
TrackTrack.current_objects

,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
9,10.193464,-3.254121,-0.711990,1.412921,0.266124,0.460025,1.337511,1.574333e+09,1,109,0.081521
10,-11.137152,15.415920,-0.911968,1.352618,0.546684,1.370234,0.708852,1.574333e+09,1,109,0.081521
11,-9.487168,19.018149,-1.403667,2.751621,1.204600,1.533254,-0.508305,1.574333e+09,1,109,0.081521
12,3.314025,27.995880,-1.186819,5.750861,2.279514,2.025559,1.530818,1.574333e+09,1,109,0.081521
13,6.557454,26.324931,-0.863639,1.568051,0.584425,1.166256,0.442498,1.574333e+09,1,109,0.081521
14,14.292184,-4.719566,-1.036344,3.694674,2.879994,1.621184,0.862217,1.574333e+09,1,130,0.127619
15,27.366480,9.388028,-0.620996,2.423051,0.704623,1.189200,-0.820607,1.574333e+09,1,130,0.127619
16,-8.244665,-16.619673,-2.642268,5.584267,0.324819,0.769942,-0.745784,1.574333e+09,1,142,0.087634
17,10.460591,-32.968130,-1.340568,3.902771,2.242807,2.530253,-1.561048,1.574333e+09,1,143,0.127471
18,2.427649,-32.079108,-0.605736,0.617645,0.512637,0.735474,-0.749255,1.574333e+09,1,143,0.127471


In [76]:
TrackTrack.current_objects

,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
20,10.109406,-3.392101,-0.743641,2.562502,0.548899,0.485338,1.465875,1.574333e+09,2,109,0.161469
21,-11.204201,15.463382,-0.987747,1.413320,0.406469,1.369057,0.777954,1.574333e+09,2,109,0.161469
22,-9.471875,18.917938,-1.344490,2.740135,1.523416,1.553007,-0.697685,1.574333e+09,2,109,0.161469
23,3.303131,27.841951,-1.221803,5.924322,2.268839,1.995784,-1.541546,1.574333e+09,2,109,0.161469
24,6.569645,26.335231,-0.868199,1.614463,0.585576,1.165900,0.497604,1.574333e+09,2,109,0.161469
25,13.713713,-5.168031,-1.059727,5.228738,2.130009,1.375944,0.587968,1.574333e+09,2,130,0.207774
26,27.402643,9.444654,-0.564636,2.396153,0.799055,1.143541,-0.822731,1.574333e+09,2,130,0.207774
27,-7.136983,-17.524426,-2.660835,6.126622,0.313524,0.823446,-0.770837,1.574333e+09,2,142,0.167598
28,0.955585,-4.324337,-1.978964,4.701532,0.519049,1.256052,-0.146043,1.574333e+09,2,142,0.167598
29,10.345271,-32.590283,-1.391791,4.925514,2.519290,2.563846,-1.493357,1.574333e+09,2,143,0.207623


In [129]:
TrackTrack.current_objects

,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
32,10.073300,-3.402564,-0.841333,2.115137,0.575810,0.713514,1.316782,1.574333e+09,3,109,0.241245
33,-11.043230,15.646841,-1.029184,1.613723,0.387612,1.470045,0.722361,1.574333e+09,3,109,0.241245
34,-9.183624,18.597415,-1.355930,1.283514,0.745171,1.878356,0.409065,1.574333e+09,3,109,0.241245
35,-10.218939,19.469227,-1.266054,3.301703,0.269861,1.258919,0.614646,1.574333e+09,3,109,0.241245
36,3.253150,27.575900,-1.217276,6.295192,2.448048,1.921330,-1.510939,1.574333e+09,3,109,0.241245
37,6.572618,26.350412,-0.868664,1.626963,0.560355,1.155252,0.573073,1.574333e+09,3,109,0.241245
38,13.600520,-5.529950,-1.147596,3.365372,1.887828,1.063619,0.237327,1.574333e+09,3,130,0.287727
39,27.363467,9.574638,-0.524335,2.298871,0.618833,0.873361,-0.769775,1.574333e+09,3,130,0.287727
40,-8.119282,-16.788614,-2.661804,4.109243,0.401864,1.188343,-0.736557,1.574333e+09,3,142,0.247703
41,10.194497,-32.303056,-1.386228,5.850100,2.598093,2.608650,-1.384838,1.574333e+09,3,143,0.287557


In [65]:
TrackTrack = Tracker([df_109,df_130,df_142,df_143])
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
0,-11.148409,15.363879,-0.941920,1.177285,0.561967,1.334849,0.631927,1.574333e+09,0,109,0.000000
1,-9.560173,18.983654,-1.377405,3.325998,0.953963,1.498572,-0.120755,1.574333e+09,0,109,0.000000
2,3.373729,28.175323,-1.134802,5.493111,1.930805,2.056836,1.514860,1.574333e+09,0,109,0.000000
3,6.546980,26.314280,-0.862299,1.506848,0.592773,1.134711,0.424306,1.574333e+09,0,109,0.000000
4,14.382445,-4.612707,-1.015361,4.811726,2.805154,1.550568,1.004918,1.574333e+09,0,130,0.047641
5,27.471277,9.154471,-0.657809,2.640507,0.653304,1.245023,-0.802791,1.574333e+09,0,130,0.047641
6,10.591985,-33.300211,-1.254730,3.106008,2.026976,2.356834,1.390347,1.574333e+09,0,143,0.047386
7,2.417817,-32.064833,-0.609711,0.641007,0.489730,0.741013,-0.615712,1.574333e+09,0,143,0.047386
8,13.233552,-5.117304,-0.707404,9.933850,2.673657,1.161759,1.067465,1.574333e+09,0,143,0.047386


{'Active': [],
 'Pending': [],
 'Inactive': [],
 'New': [ID 0 - TimeNew 1
   - Last coords (-11.148409,15.363879,-0.941920),
  ID 1 - TimeNew 1
   - Last coords (-9.560173,18.983654,-1.377405),
  ID 2 - TimeNew 1
   - Last coords (3.373729,28.175323,-1.134802),
  ID 3 - TimeNew 1
   - Last coords (6.546980,26.314280,-0.862299),
  ID 4 - TimeNew 1
   - Last coords (14.382445,-4.612707,-1.015361),
  ID 5 - TimeNew 1
   - Last coords (27.471277,9.154471,-0.657809),
  ID 6 - TimeNew 1
   - Last coords (10.591985,-33.300211,-1.254730),
  ID 7 - TimeNew 1
   - Last coords (2.417817,-32.064833,-0.609711),
  ID 8 - TimeNew 1
   - Last coords (13.233552,-5.117304,-0.707404)],
 'Removed': []}

In [66]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

Iou associations
[0.87  0.695 0.84  0.955 0.815 0.715 0.81  0.95  0.72 ]
[ 1  2  3  4  5  6  8  9 10]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
9,10.193464,-3.254121,-0.711990,1.412921,0.266124,0.460025,1.337511,1.574333e+09,1,109,0.081521
10,-11.137152,15.415920,-0.911968,1.352618,0.546684,1.370234,0.708852,1.574333e+09,1,109,0.081521
11,-9.487168,19.018149,-1.403667,2.751621,1.204600,1.533254,-0.508305,1.574333e+09,1,109,0.081521
12,3.314025,27.995880,-1.186819,5.750861,2.279514,2.025559,1.530818,1.574333e+09,1,109,0.081521
13,6.557454,26.324931,-0.863639,1.568051,0.584425,1.166256,0.442498,1.574333e+09,1,109,0.081521
14,14.292184,-4.719566,-1.036344,3.694674,2.879994,1.621184,0.862217,1.574333e+09,1,130,0.127619
15,27.366480,9.388028,-0.620996,2.423051,0.704623,1.189200,-0.820607,1.574333e+09,1,130,0.127619
16,-8.244665,-16.619673,-2.642268,5.584267,0.324819,0.769942,-0.745784,1.574333e+09,1,142,0.087634
17,10.460591,-32.968130,-1.340568,3.902771,2.242807,2.530253,-1.561048,1.574333e+09,1,143,0.127471
18,2.427649,-32.079108,-0.605736,0.617645,0.512637,0.735474,-0.749255,1.574333e+09,1,143,0.127471


{'Active': [],
 'Pending': [],
 'Inactive': [],
 'New': [ID 0 - TimeNew 2
   - Last coords (-11.137152,15.415920,-0.911968),
  ID 1 - TimeNew 2
   - Last coords (-9.487168,19.018149,-1.403667),
  ID 2 - TimeNew 2
   - Last coords (3.314025,27.995880,-1.186819),
  ID 3 - TimeNew 2
   - Last coords (6.557454,26.324931,-0.863639),
  ID 4 - TimeNew 2
   - Last coords (14.292184,-4.719566,-1.036344),
  ID 5 - TimeNew 2
   - Last coords (27.366480,9.388028,-0.620996),
  ID 6 - TimeNew 2
   - Last coords (10.460591,-32.968130,-1.340568),
  ID 7 - TimeNew 2
   - Last coords (2.427649,-32.079108,-0.605736),
  ID 8 - TimeNew 2
   - Last coords (13.044299,-5.497306,-0.693706),
  ID 9 - TimeNew 1
   - Last coords (10.193464,-3.254121,-0.711990),
  ID 10 - TimeNew 1
   - Last coords (-8.244665,-16.619673,-2.642268)],
 'Removed': []}

In [67]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

Iou associations
[0.73  0.83  0.925 0.95  0.69  0.85  0.82  0.955 0.59  0.6   0.475]
[ 1  2  3  4  5  6  9 10 11  0  7]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
20,10.109406,-3.392101,-0.743641,2.562502,0.548899,0.485338,1.465875,1.574333e+09,2,109,0.161469
21,-11.204201,15.463382,-0.987747,1.413320,0.406469,1.369057,0.777954,1.574333e+09,2,109,0.161469
22,-9.471875,18.917938,-1.344490,2.740135,1.523416,1.553007,-0.697685,1.574333e+09,2,109,0.161469
23,3.303131,27.841951,-1.221803,5.924322,2.268839,1.995784,-1.541546,1.574333e+09,2,109,0.161469
24,6.569645,26.335231,-0.868199,1.614463,0.585576,1.165900,0.497604,1.574333e+09,2,109,0.161469
25,13.713713,-5.168031,-1.059727,5.228738,2.130009,1.375944,0.587968,1.574333e+09,2,130,0.207774
26,27.402643,9.444654,-0.564636,2.396153,0.799055,1.143541,-0.822731,1.574333e+09,2,130,0.207774
27,-7.136983,-17.524426,-2.660835,6.126622,0.313524,0.823446,-0.770837,1.574333e+09,2,142,0.167598
28,0.955585,-4.324337,-1.978964,4.701532,0.519049,1.256052,-0.146043,1.574333e+09,2,142,0.167598
29,10.345271,-32.590283,-1.391791,4.925514,2.519290,2.563846,-1.493357,1.574333e+09,2,143,0.207623


{'Active': [],
 'Pending': [],
 'Inactive': [],
 'New': [ID 0 - TimeNew 3
   - Last coords (-11.204201,15.463382,-0.987747),
  ID 1 - TimeNew 3
   - Last coords (-9.471875,18.917938,-1.344490),
  ID 2 - TimeNew 3
   - Last coords (3.303131,27.841951,-1.221803),
  ID 3 - TimeNew 3
   - Last coords (6.569645,26.335231,-0.868199),
  ID 4 - TimeNew 3
   - Last coords (13.713713,-5.168031,-1.059727),
  ID 5 - TimeNew 3
   - Last coords (27.402643,9.444654,-0.564636),
  ID 6 - TimeNew 3
   - Last coords (10.345271,-32.590283,-1.391791),
  ID 7 - TimeNew 3
   - Last coords (2.435785,-32.083477,-0.609795),
  ID 8 - TimeNew 3
   - Last coords (12.634190,-6.548765,-0.752718),
  ID 9 - TimeNew 2
   - Last coords (10.109406,-3.392101,-0.743641),
  ID 10 - TimeNew 2
   - Last coords (-7.136983,-17.524426,-2.660835),
  ID 11 - TimeNew 1
   - Last coords (0.955585,-4.324337,-1.978964)],
 'Removed': []}

In [68]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

Iou associations
[0.76  0.51  0.895 0.93  0.625 0.74  0.84  0.905 0.885 0.635 0.405 0.   ]
[ 1  2  4  5  6  7  9 10 11  0  8 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
32,10.073300,-3.402564,-0.841333,2.115137,0.575810,0.713514,1.316782,1.574333e+09,3,109,0.241245
33,-11.043230,15.646841,-1.029184,1.613723,0.387612,1.470045,0.722361,1.574333e+09,3,109,0.241245
34,-9.183624,18.597415,-1.355930,1.283514,0.745171,1.878356,0.409065,1.574333e+09,3,109,0.241245
35,-10.218939,19.469227,-1.266054,3.301703,0.269861,1.258919,0.614646,1.574333e+09,3,109,0.241245
36,3.253150,27.575900,-1.217276,6.295192,2.448048,1.921330,-1.510939,1.574333e+09,3,109,0.241245
37,6.572618,26.350412,-0.868664,1.626963,0.560355,1.155252,0.573073,1.574333e+09,3,109,0.241245
38,13.600520,-5.529950,-1.147596,3.365372,1.887828,1.063619,0.237327,1.574333e+09,3,130,0.287727
39,27.363467,9.574638,-0.524335,2.298871,0.618833,0.873361,-0.769775,1.574333e+09,3,130,0.287727
40,-8.119282,-16.788614,-2.661804,4.109243,0.401864,1.188343,-0.736557,1.574333e+09,3,142,0.247703
41,10.194497,-32.303056,-1.386228,5.850100,2.598093,2.608650,-1.384838,1.574333e+09,3,143,0.287557


{'Active': [],
 'Pending': [],
 'Inactive': [],
 'New': [ID 0 - TimeNew 4
   - Last coords (-11.043230,15.646841,-1.029184),
  ID 1 - TimeNew 4
   - Last coords (-9.183624,18.597415,-1.355930),
  ID 2 - TimeNew 4
   - Last coords (3.253150,27.575900,-1.217276),
  ID 3 - TimeNew 4
   - Last coords (6.572618,26.350412,-0.868664),
  ID 4 - TimeNew 4
   - Last coords (13.600520,-5.529950,-1.147596),
  ID 5 - TimeNew 4
   - Last coords (27.363467,9.574638,-0.524335),
  ID 6 - TimeNew 4
   - Last coords (10.194497,-32.303056,-1.386228),
  ID 7 - TimeNew 4
   - Last coords (2.442763,-32.097151,-0.607468),
  ID 8 - TimeNew 4
   - Last coords (12.451625,-6.807836,-0.733931),
  ID 9 - TimeNew 3
   - Last coords (10.073300,-3.402564,-0.841333),
  ID 10 - TimeNew 3
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 12 - TimeNew 1
   - Last coords (-10.218939,19.469227,-1.266054)],
 'Removed': [ID 11 - TimeNew 1
   - Last coords (0.955585,-4.324337,-1.978964)]}

In [69]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

Iou associations
[0.835 0.76  0.905 0.905 0.725 0.665 0.845 0.945 0.865 0.58  0.    0.665]
[ 1  2  4  6  7  9 12 13 14  0 -1  3]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
44,10.065049,-4.124774,-0.858143,3.049376,0.632370,0.646122,1.547729,1.574333e+09,4,109,0.320305
45,-11.094020,15.663410,-0.992140,1.545281,0.490576,1.329137,0.708912,1.574333e+09,4,109,0.320305
46,-9.246327,18.480157,-1.370194,1.436779,0.704693,1.725794,0.366784,1.574333e+09,4,109,0.320305
47,-9.778824,19.738528,-1.356301,2.301618,0.378689,1.125864,0.619657,1.574333e+09,4,109,0.320305
48,3.250484,27.179847,-1.187397,6.568525,2.504693,1.925749,-1.525559,1.574333e+09,4,109,0.320305
49,0.944654,25.487162,-0.740430,1.428372,0.782623,0.750498,0.724903,1.574333e+09,4,109,0.320305
50,6.578455,26.354515,-0.851427,1.620492,0.566774,1.145562,0.654950,1.574333e+09,4,109,0.320305
51,13.448071,-5.935414,-1.136619,2.943590,1.750460,1.112790,0.282009,1.574333e+09,4,130,0.367572
52,18.884096,-5.348304,-1.834904,1.304201,0.174620,0.948919,0.615424,1.574333e+09,4,130,0.367572
53,27.342745,9.669458,-0.558172,1.376027,0.547921,1.003843,-0.703525,1.574333e+09,4,130,0.367572


{'Active': [],
 'Pending': [],
 'Inactive': [],
 'New': [ID 0 - TimeNew 5
   - Last coords (-11.094020,15.663410,-0.992140),
  ID 1 - TimeNew 5
   - Last coords (-9.246327,18.480157,-1.370194),
  ID 2 - TimeNew 5
   - Last coords (3.250484,27.179847,-1.187397),
  ID 3 - TimeNew 5
   - Last coords (6.578455,26.354515,-0.851427),
  ID 4 - TimeNew 5
   - Last coords (13.448071,-5.935414,-1.136619),
  ID 5 - TimeNew 5
   - Last coords (27.342745,9.669458,-0.558172),
  ID 6 - TimeNew 5
   - Last coords (10.074895,-32.100031,-1.428653),
  ID 7 - TimeNew 5
   - Last coords (2.435988,-32.107087,-0.600418),
  ID 8 - TimeNew 5
   - Last coords (12.295501,-7.300857,-0.751893),
  ID 9 - TimeNew 4
   - Last coords (10.065049,-4.124774,-0.858143),
  ID 10 - TimeNew 4
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 12 - TimeNew 2
   - Last coords (-9.778824,19.738528,-1.356301),
  ID 13 - TimeNew 1
   - Last coords (0.944654,25.487162,-0.740430),
  ID 14 - TimeNew 1
   - Last coords (18.88409

In [70]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

Iou associations
[0.75  0.795 0.745 0.935 0.635 0.48  0.875 0.935 0.785 0.555 0.    0.795
 0.    0.    0.    0.765]
[ 1  2  4  5  7  8 10 11 12  0 -1  3 -1 -1 -1  9]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
59,9.969764,-5.600120,-0.881406,3.652781,0.626077,0.679944,1.438288,1.574333e+09,5,109,0.400123
60,-11.163822,15.705071,-0.987490,1.702332,0.401186,1.166153,0.686620,1.574333e+09,5,109,0.400123
61,-9.251641,18.383025,-1.365100,1.253955,0.661718,1.629647,0.235827,1.574333e+09,5,109,0.400123
62,-9.608339,19.818089,-1.288390,2.216803,0.434420,1.318860,0.668512,1.574333e+09,5,109,0.400123
63,3.161106,26.703710,-1.149463,6.794674,3.309993,1.916772,1.506635,1.574333e+09,5,109,0.400123
64,6.568316,26.364439,-0.842710,1.644674,0.570360,1.134798,0.721171,1.574333e+09,5,109,0.400123
65,12.086696,-9.833602,-0.782232,2.601114,1.059756,0.650701,0.706308,1.574333e+09,5,130,0.447638
66,13.288051,-6.208939,-1.104970,2.781427,0.840798,1.191733,-0.057430,1.574333e+09,5,130,0.447638
67,27.201655,9.868030,-0.440152,1.302547,0.771170,0.410919,-1.092147,1.574333e+09,5,130,0.447638
68,-28.240649,5.021160,-1.058012,0.662643,0.448960,2.105271,-0.676414,1.574333e+09,5,142,0.407629


{'Active': [],
 'Pending': [],
 'Inactive': [],
 'New': [ID 0 - TimeNew 6
   - Last coords (-11.163822,15.705071,-0.987490),
  ID 1 - TimeNew 6
   - Last coords (-9.251641,18.383025,-1.365100),
  ID 2 - TimeNew 6
   - Last coords (3.161106,26.703710,-1.149463),
  ID 3 - TimeNew 6
   - Last coords (6.568316,26.364439,-0.842710),
  ID 4 - TimeNew 6
   - Last coords (13.288051,-6.208939,-1.104970),
  ID 5 - TimeNew 6
   - Last coords (27.201655,9.868030,-0.440152),
  ID 6 - TimeNew 6
   - Last coords (9.941542,-31.729394,-1.404740),
  ID 7 - TimeNew 6
   - Last coords (2.435993,-32.115778,-0.603797),
  ID 8 - TimeNew 6
   - Last coords (12.083010,-7.980987,-0.745801),
  ID 9 - TimeNew 5
   - Last coords (9.969764,-5.600120,-0.881406),
  ID 10 - TimeNew 5
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 12 - TimeNew 3
   - Last coords (-9.608339,19.818089,-1.288390),
  ID 16 - TimeNew 2
   - Last coords (-28.240649,5.021160,-1.058012),
  ID 17 - TimeNew 1
   - Last coords (12.086696

In [71]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

#1 is the prediction used for nn when uding KF?
# if yes, try to tune the noises cause it looks like there is too much

Iou associations
[0.65  0.62  0.93  0.96  0.345 0.65  0.89  0.975 0.8   0.47  0.    0.8
 0.82  0.565]
[ 1  2  4  5 13  8 11 12 13  0 -1  3 10  6]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
72,10.032480,-2.900468,-0.925043,7.649287,1.099446,1.205971,1.445841,1.574333e+09,6,109,0.481813
73,-11.288490,15.691054,-1.057446,2.206232,0.351428,1.191374,0.689989,1.574333e+09,6,109,0.481813
74,-9.383795,18.202067,-1.365403,1.234077,0.617000,1.686681,0.225427,1.574333e+09,6,109,0.481813
75,-9.557422,19.904422,-1.228670,1.848318,0.418641,1.442104,0.734870,1.574333e+09,6,109,0.481813
76,3.123269,26.321674,-1.134761,6.904570,3.626939,1.893255,1.477850,1.574333e+09,6,109,0.481813
77,6.565439,26.382995,-0.834830,1.653252,0.542498,1.122467,0.777023,1.574333e+09,6,109,0.481813
78,12.203092,-9.949252,-0.854266,4.614563,1.019891,1.356051,0.876714,1.574333e+09,6,130,0.527659
79,12.583373,-6.610295,-0.942711,2.579120,0.630177,0.884923,0.115628,1.574333e+09,6,130,0.527659
80,27.209659,9.810051,-0.392463,1.174977,0.543666,0.209451,-0.667013,1.574333e+09,6,130,0.527659
81,-1.046515,-6.175298,-1.803788,3.558082,0.725090,0.593218,-0.315269,1.574333e+09,6,142,0.487756


{'Active': [],
 'Pending': [],
 'Inactive': [],
 'New': [ID 0 - TimeNew 7
   - Last coords (-11.288490,15.691054,-1.057446),
  ID 1 - TimeNew 7
   - Last coords (-9.383795,18.202067,-1.365403),
  ID 2 - TimeNew 7
   - Last coords (3.123269,26.321674,-1.134761),
  ID 3 - TimeNew 7
   - Last coords (6.565439,26.382995,-0.834830),
  ID 4 - TimeNew 7
   - Last coords (11.893694,-8.940502,-0.793929),
  ID 5 - TimeNew 7
   - Last coords (27.209659,9.810051,-0.392463),
  ID 6 - TimeNew 7
   - Last coords (9.770702,-31.270721,-1.373968),
  ID 7 - TimeNew 7
   - Last coords (2.442384,-32.120741,-0.603769),
  ID 8 - TimeNew 7
   - Last coords (11.893694,-8.940502,-0.793929),
  ID 9 - TimeNew 6
   - Last coords (10.032480,-2.900468,-0.925043),
  ID 10 - TimeNew 6
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 12 - TimeNew 4
   - Last coords (-9.557422,19.904422,-1.228670),
  ID 16 - TimeNew 3
   - Last coords (-28.168700,5.035866,-1.074671),
  ID 17 - TimeNew 2
   - Last coords (12.20309

In [72]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

Iou associations
[0.745 0.795 0.9   0.925 0.75  0.    0.84  0.91  0.72  0.725 0.    0.64
 0.84  0.615 0.    0.3  ]
[ 1  2  4  5 13 -1 12 11 13  0 -1  3 10  6 -1  8]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
86,9.965130,-3.698896,-0.862709,9.526631,1.172517,1.003382,1.507217,1.574333e+09,7,109,0.560056
87,-11.014090,15.942268,-1.113405,2.132019,0.354482,1.242151,0.701535,1.574333e+09,7,109,0.560056
88,-9.299302,18.152167,-1.361036,1.230536,0.588501,1.851735,0.202899,1.574333e+09,7,109,0.560056
89,-9.477916,20.063715,-1.252146,1.313803,0.492111,1.542235,0.721651,1.574333e+09,7,109,0.560056
90,3.134519,25.947233,-1.132872,6.844868,3.537909,1.865840,1.501953,1.574333e+09,7,109,0.560056
91,6.576440,26.408238,-0.815346,1.609916,0.510466,1.080364,0.861228,1.574333e+09,7,109,0.560056
92,12.447078,-9.985398,-0.940990,5.724873,2.127393,1.437357,0.539637,1.574333e+09,7,130,0.607710
93,18.614534,-5.497623,-2.120894,1.842953,0.193958,0.572443,0.597929,1.574333e+09,7,130,0.607710
94,-2.321130,-5.774212,-2.068198,4.773581,0.526979,0.660932,-0.109805,1.574333e+09,7,142,0.567675
95,-23.488230,0.048057,-0.717006,1.352411,0.279099,1.276310,-0.801580,1.574333e+09,7,142,0.567675


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.671658,-9.986991,-0.792898),
  ID 7 - TimeNew 8
   - Last coords (2.458079,-32.130815,-0.609073),
  ID 6 - TimeNew 8
   - Last coords (9.583402,-30.779545,-1.340911),
  ID 4 - TimeNew 8
   - Last coords (11.671658,-9.986991,-0.792898),
  ID 3 - TimeNew 8
   - Last coords (6.576440,26.408238,-0.815346),
  ID 2 - TimeNew 8
   - Last coords (3.134519,25.947233,-1.132872),
  ID 1 - TimeNew 8
   - Last coords (-9.299302,18.152167,-1.361036),
  ID 0 - TimeNew 8
   - Last coords (-11.014090,15.942268,-1.113405)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 8
   - Last coords (27.209659,9.810051,-0.392463),
  ID 9 - TimeNew 7
   - Last coords (9.965130,-3.698896,-0.862709),
  ID 10 - TimeNew 7
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 12 - TimeNew 5
   - Last coords (-9.477916,20.063715,-1.252146),
  ID 16 - TimeNew 4
   - Last coords (-28.085358,5.056278,-1.071124),
  ID 17 - TimeNew 3
   - Last coords (12.447078,-9.

In [73]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.27579567 0.0184298  0.64563814 0.27579567 0.01595812 0.35070306
 0.21035118 0.14645903]
[11  9 10 11  5  4  2  1]
Iou associations
[0.    0.71  0.    0.815 0.815 0.54  0.    0.    0.595]
[-1  0 -1  1  4  2 -1 -1  3]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
100,9.883868,-3.682064,-0.972213,7.057720,1.355886,1.212415,1.516849,1.574333e+09,8,109,0.641159
101,-10.931569,16.055569,-1.155870,2.026066,0.369320,1.313004,0.711914,1.574333e+09,8,109,0.641159
102,-9.447539,18.002991,-1.356579,1.319826,0.654763,1.966696,0.403460,1.574333e+09,8,109,0.641159
103,-9.478577,20.156246,-1.246032,1.344110,0.476570,1.524768,0.707764,1.574333e+09,8,109,0.641159
104,3.168451,25.598200,-1.136967,6.798623,3.533242,1.846936,1.532719,1.574333e+09,8,109,0.641159
105,6.565841,26.416818,-0.807057,1.648070,0.480915,1.144515,0.892072,1.574333e+09,8,109,0.641159
106,13.499119,-9.621376,-1.164734,7.827864,1.346491,0.759593,0.358815,1.574333e+09,8,130,0.687707
107,-23.451890,0.030349,-1.077770,1.561390,0.545271,2.304060,-0.850194,1.574333e+09,8,142,0.647689
108,-27.988617,5.047180,-1.126286,0.770187,0.492467,1.913658,-0.357601,1.574333e+09,8,142,0.647689
109,2.442369,-32.125719,-0.600895,0.613001,0.477670,0.730225,-0.776090,1.574333e+09,8,143,0.687491


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.619188,-10.256946,-0.772046),
  ID 7 - TimeNew 8
   - Last coords (2.442369,-32.125719,-0.600895),
  ID 6 - TimeNew 8
   - Last coords (9.329469,-30.187199,-1.302279),
  ID 4 - TimeNew 8
   - Last coords (11.619188,-10.256946,-0.772046),
  ID 3 - TimeNew 8
   - Last coords (6.565841,26.416818,-0.807057),
  ID 2 - TimeNew 8
   - Last coords (3.168451,25.598200,-1.136967),
  ID 1 - TimeNew 8
   - Last coords (-9.447539,18.002991,-1.356579),
  ID 0 - TimeNew 8
   - Last coords (-10.931569,16.055569,-1.155870),
  ID 9 - TimeNew 8
   - Last coords (9.883868,-3.682064,-0.972213)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 9
   - Last coords (27.209659,9.810051,-0.392463),
  ID 10 - TimeNew 8
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 12 - TimeNew 6
   - Last coords (-9.478577,20.156246,-1.246032),
  ID 16 - TimeNew 5
   - Last coords (-27.988617,5.047180,-1.126286),
  ID 17 - TimeNew 4
   - Last coords (13.499119,-

In [74]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.94240771 0.01680901 0.66715826 0.94240771 0.04020607 0.37194539
 0.14367165 0.19692091 0.11246574]
[11  9 10 11  5  4  2  1  0]
Iou associations
[0.    0.    0.715 0.75  0.475 0.    0.615]
[-1 -1  0  3  1 -1  2]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
112,9.931915,-3.597753,-1.029061,7.271231,1.408025,1.273710,1.476447,1.574333e+09,9,109,0.720403
113,-10.779031,16.179397,-1.142579,2.046952,0.381126,1.308598,0.658676,1.574333e+09,9,109,0.720403
114,-9.443646,17.910826,-1.466725,1.231907,0.696082,1.644936,0.357000,1.574333e+09,9,109,0.720403
115,-9.246519,20.296534,-1.240268,1.132613,0.443839,1.474238,0.796755,1.574333e+09,9,109,0.720403
116,3.186254,25.226757,-1.129433,6.763483,3.652780,1.826552,-1.567229,1.574333e+09,9,109,0.720403
117,6.526759,26.408058,-0.810580,1.768505,0.506868,1.221522,0.887649,1.574333e+09,9,109,0.720403
118,13.409157,-9.918548,-1.179945,9.911528,0.869491,1.172247,0.459548,1.574333e+09,9,130,0.767725
119,-23.356689,-0.168447,-1.018426,1.421981,0.608559,2.158488,-1.184175,1.574333e+09,9,142,0.727659
120,-27.888761,5.055795,-1.151217,0.782299,0.486171,1.776582,-0.471140,1.574333e+09,9,142,0.727659
121,2.455598,-32.136083,-0.600558,0.673821,0.481109,0.731176,-0.731758,1.574333e+09,9,143,0.767704


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.489692,-11.190168,-0.750592),
  ID 7 - TimeNew 8
   - Last coords (2.455598,-32.136083,-0.600558),
  ID 6 - TimeNew 8
   - Last coords (9.048790,-29.582864,-1.269132),
  ID 4 - TimeNew 8
   - Last coords (11.489692,-11.190168,-0.750592),
  ID 3 - TimeNew 8
   - Last coords (6.526759,26.408058,-0.810580),
  ID 2 - TimeNew 8
   - Last coords (3.186254,25.226757,-1.129433),
  ID 1 - TimeNew 8
   - Last coords (-9.443646,17.910826,-1.466725),
  ID 0 - TimeNew 8
   - Last coords (-10.779031,16.179397,-1.142579),
  ID 9 - TimeNew 8
   - Last coords (9.931915,-3.597753,-1.029061)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 10
   - Last coords (27.209659,9.810051,-0.392463),
  ID 10 - TimeNew 9
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 12 - TimeNew 7
   - Last coords (-9.246519,20.296534,-1.240268),
  ID 16 - TimeNew 6
   - Last coords (-27.888761,5.055795,-1.151217),
  ID 17 - TimeNew 5
   - Last coords (13.409157,

In [75]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.44037076 0.01355429 0.67990617 0.44037076 0.01349291 0.31219179
 0.65748516 1.55335174 0.45507326]
[12  9 10 12  4  3  1  1  0]
Iou associations
[0.   0.   0.7  0.76 0.57 0.   0.81]
[-1 -1  0  4  1 -1  3]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
124,9.969044,-4.049757,-1.066552,6.516701,1.010123,1.356889,1.398049,1.574333e+09,10,109,0.799993
125,-9.850619,17.405932,-1.358369,4.567475,0.744613,1.748858,0.854412,1.574333e+09,10,109,0.799993
126,-9.221202,20.342044,-1.259286,1.264449,0.632578,1.279195,0.444142,1.574333e+09,10,109,0.799993
127,3.250242,24.921307,-1.137783,6.586164,3.248729,1.790128,-1.437991,1.574333e+09,10,109,0.799993
128,6.516425,26.414938,-0.815866,1.892317,0.525924,1.234372,0.892525,1.574333e+09,10,109,0.799993
129,14.734831,-9.362089,-1.383167,7.694298,0.481848,0.705104,0.498441,1.574333e+09,10,130,0.847682
130,11.304579,-9.656147,-1.176606,2.065800,0.602588,0.712424,0.520836,1.574333e+09,10,130,0.847682
131,-23.280203,-0.383727,-1.012278,1.749876,0.567413,2.119915,-1.320172,1.574333e+09,10,142,0.807800
132,-27.813777,5.061416,-1.128966,0.732364,0.463409,1.901375,-0.564198,1.574333e+09,10,142,0.807800
133,2.446905,-32.136449,-0.590165,0.669605,0.468362,0.691076,-0.957352,1.574333e+09,10,143,0.847512


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.463383,-11.628716,-0.780745),
  ID 7 - TimeNew 8
   - Last coords (2.446905,-32.136449,-0.590165),
  ID 6 - TimeNew 8
   - Last coords (8.759944,-28.968128,-1.238463),
  ID 4 - TimeNew 8
   - Last coords (11.463383,-11.628716,-0.780745),
  ID 3 - TimeNew 8
   - Last coords (6.516425,26.414938,-0.815866),
  ID 2 - TimeNew 8
   - Last coords (3.250242,24.921307,-1.137783),
  ID 1 - TimeNew 8
   - Last coords (-9.850619,17.405932,-1.358369),
  ID 0 - TimeNew 8
   - Last coords (-9.850619,17.405932,-1.358369),
  ID 9 - TimeNew 8
   - Last coords (9.969044,-4.049757,-1.066552),
  ID 12 - TimeNew 8
   - Last coords (-9.221202,20.342044,-1.259286)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 11
   - Last coords (27.209659,9.810051,-0.392463),
  ID 10 - TimeNew 10
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 16 - TimeNew 7
   - Last coords (-27.813777,5.061416,-1.128966),
  ID 17 - TimeNew 6
   - Last coords (14.734831,

In [76]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.15571059 0.01672576 0.65277945 0.15571059 0.01022497 0.35820773
 0.2265876  0.2265876  0.80570241 0.14443571]
[12  9 10 12  4  3  1  1  0  2]
Iou associations
[0.    0.    0.695 0.515 0.    0.725 0.225]
[-1 -1  3  1 -1  2  0]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
137,9.955493,-4.854411,-1.027766,4.941556,1.475853,1.323062,1.537869,1.574333e+09,11,109,0.879650
138,-10.003746,17.242004,-1.326403,4.631711,0.510396,1.689014,0.828565,1.574333e+09,11,109,0.879650
139,-9.116385,20.434152,-1.296584,1.203279,0.498124,1.332847,0.576350,1.574333e+09,11,109,0.879650
140,3.293259,24.565936,-1.124615,6.542991,3.302180,1.783326,-1.406588,1.574333e+09,11,109,0.879650
141,6.515020,26.424335,-0.819643,1.903549,0.528134,1.230930,0.926463,1.574333e+09,11,109,0.879650
142,12.027395,-9.592610,-1.078782,2.906456,0.721475,1.089822,0.721904,1.574333e+09,11,130,0.927611
143,15.576758,-8.907120,-1.479553,3.502024,0.554573,1.024484,0.553748,1.574333e+09,11,130,0.927611
144,-23.198601,-0.721678,-0.984925,1.824051,0.581350,2.151748,-1.416924,1.574333e+09,11,142,0.887704
145,-27.702019,5.096290,-1.040127,0.714395,0.505195,1.766613,-0.756200,1.574333e+09,11,142,0.887704
146,2.455661,-32.145481,-0.601188,0.602597,0.484905,0.728048,-0.388468,1.574333e+09,11,143,0.927557


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.481590,-11.783355,-0.779717),
  ID 7 - TimeNew 8
   - Last coords (2.455661,-32.145481,-0.601188),
  ID 6 - TimeNew 8
   - Last coords (8.497302,-28.370640,-1.226319),
  ID 4 - TimeNew 8
   - Last coords (11.481590,-11.783355,-0.779717),
  ID 3 - TimeNew 8
   - Last coords (6.515020,26.424335,-0.819643),
  ID 2 - TimeNew 8
   - Last coords (3.293259,24.565936,-1.124615),
  ID 1 - TimeNew 8
   - Last coords (-10.003746,17.242004,-1.326403),
  ID 0 - TimeNew 8
   - Last coords (-10.003746,17.242004,-1.326403),
  ID 9 - TimeNew 8
   - Last coords (9.955493,-4.854411,-1.027766),
  ID 12 - TimeNew 8
   - Last coords (-9.116385,20.434152,-1.296584),
  ID 16 - TimeNew 8
   - Last coords (-27.702019,5.096290,-1.040127)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 12
   - Last coords (27.209659,9.810051,-0.392463),
  ID 10 - TimeNew 11
   - Last coords (-8.119282,-16.788614,-2.661804),
  ID 17 - TimeNew 7
   - Last coords (15.57675

In [77]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[1.02416665 0.02012513 0.71143906 1.02416665 0.01678828 0.42819272
 0.07839367 0.07839367 0.15149195 0.18474994 0.1143628 ]
[12 10 11 12  4  3  1  1  0  2  8]
Iou associations
[0.    0.    0.645 0.    0.755 0.6  ]
[-1 -1  1 -1  2  0]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
150,9.868580,-4.969390,-1.074411,6.172986,1.393867,1.331525,-1.554661,1.574333e+09,12,109,0.960091
151,-9.931259,17.230798,-1.354073,3.902932,0.625007,1.668542,0.802897,1.574333e+09,12,109,0.960091
152,-8.953990,20.519725,-1.317508,1.321721,0.362599,1.506076,0.597862,1.574333e+09,12,109,0.960091
153,3.348853,24.141475,-1.115061,6.640288,3.157241,1.778410,-1.382300,1.574333e+09,12,109,0.960091
154,6.518266,26.439208,-0.826723,1.933607,0.522651,1.259300,0.947942,1.574333e+09,12,109,0.960091
155,12.232365,-9.841371,-1.060979,2.199721,1.214231,1.135039,0.641337,1.574333e+09,12,130,1.007570
156,15.479898,-8.984684,-1.557284,1.572337,0.459810,1.144665,0.513338,1.574333e+09,12,130,1.007570
157,-23.115446,-1.046180,-0.971332,1.872725,0.597913,2.202248,-1.442860,1.574333e+09,12,142,0.967615
158,-27.595317,5.120160,-1.006605,0.718738,0.513002,1.958000,-0.675436,1.574333e+09,12,142,0.967615
159,2.643024,19.997574,-0.410586,4.040663,0.500139,0.336629,0.484418,1.574333e+09,12,142,0.967615


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.419454,-12.804906,-0.818333),
  ID 7 - TimeNew 8
   - Last coords (2.440232,-32.133240,-0.597050),
  ID 6 - TimeNew 8
   - Last coords (8.204115,-27.724109,-1.273065),
  ID 4 - TimeNew 8
   - Last coords (11.419454,-12.804906,-0.818333),
  ID 3 - TimeNew 8
   - Last coords (6.518266,26.439208,-0.826723),
  ID 2 - TimeNew 8
   - Last coords (3.348853,24.141475,-1.115061),
  ID 1 - TimeNew 8
   - Last coords (-9.931259,17.230798,-1.354073),
  ID 0 - TimeNew 8
   - Last coords (-9.931259,17.230798,-1.354073),
  ID 9 - TimeNew 8
   - Last coords (9.868580,-4.969390,-1.074411),
  ID 12 - TimeNew 8
   - Last coords (-8.953990,20.519725,-1.317508),
  ID 16 - TimeNew 8
   - Last coords (-27.595317,5.120160,-1.006605),
  ID 17 - TimeNew 8
   - Last coords (15.479898,-8.984684,-1.557284)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 13
   - Last coords (27.209659,9.810051,-0.392463),
  ID 10 - TimeNew 12
   - Last coords (-8.119282,-

In [78]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.68537867 0.02211463 0.63754068 0.68537867 0.01424645 0.38668408
 0.23115002 0.23115002 0.85752859 0.13344463 0.17711837 2.84693131]
[10  8  9 10  4  3  1  1  0  2  7  5]
Iou associations
[0.   0.   0.   0.65 0.  ]
[-1 -1 -1  0 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
163,9.863140,-5.826868,-1.066785,6.083501,1.118311,1.114126,1.543121,1.574333e+09,13,109,1.040221
164,-10.107218,17.083994,-1.323790,2.858363,0.494727,1.715168,0.844118,1.574333e+09,13,109,1.040221
165,-8.842779,20.585040,-1.283250,1.454084,0.336379,1.457928,0.601398,1.574333e+09,13,109,1.040221
166,3.393075,23.757761,-1.096825,6.605223,3.082330,1.823604,-1.368381,1.574333e+09,13,109,1.040221
167,6.512167,26.448413,-0.835725,1.981931,0.562826,1.284312,0.950963,1.574333e+09,13,109,1.040221
168,12.865232,-10.006916,-1.084438,8.378524,0.990139,1.487455,0.357828,1.574333e+09,13,130,1.087488
169,-22.991240,-1.434491,-0.975848,1.877628,0.618350,2.191434,-1.406603,1.574333e+09,13,142,1.047605
170,-27.503309,5.116220,-0.855311,0.899128,0.462423,2.031086,-0.701827,1.574333e+09,13,142,1.047605
171,2.424692,-32.125885,-0.583140,0.607073,0.475934,0.689830,-0.759686,1.574333e+09,13,143,1.087443
172,7.934852,-27.146317,-1.283630,6.489674,2.708221,2.682841,-0.804002,1.574333e+09,13,143,1.087443


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.427502,-13.490137,-0.806607),
  ID 7 - TimeNew 8
   - Last coords (2.424692,-32.125885,-0.583140),
  ID 6 - TimeNew 8
   - Last coords (7.934852,-27.146317,-1.283630),
  ID 4 - TimeNew 8
   - Last coords (11.427502,-13.490137,-0.806607),
  ID 3 - TimeNew 8
   - Last coords (6.512167,26.448413,-0.835725),
  ID 2 - TimeNew 8
   - Last coords (3.393075,23.757761,-1.096825),
  ID 1 - TimeNew 8
   - Last coords (-10.107218,17.083994,-1.323790),
  ID 0 - TimeNew 8
   - Last coords (-10.107218,17.083994,-1.323790),
  ID 9 - TimeNew 8
   - Last coords (9.863140,-5.826868,-1.066785),
  ID 12 - TimeNew 8
   - Last coords (-8.842779,20.585040,-1.283250),
  ID 16 - TimeNew 8
   - Last coords (-27.503309,5.116220,-0.855311),
  ID 17 - TimeNew 8
   - Last coords (12.865232,-10.006916,-1.084438)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 14
   - Last coords (27.209659,9.810051,-0.392463),
  ID 10 - TimeNew 13
   - Last coords (-8.11928

In [79]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.25199697 0.02340281 0.69922748 0.25199697 0.02729583 0.43437345
 0.0464679  0.0464679  0.2397969  0.19776461 0.14286695 0.44929843]
[12 10 11 12  4  3  1  1  0  2  8  5]
Iou associations
[0.    0.    0.    0.745 0.   ]
[-1 -1 -1  1 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
174,9.886663,-6.064847,-1.084531,4.965305,1.137281,1.127290,1.526371,1.574333e+09,14,109,1.122607
175,-10.067497,17.093422,-1.345984,2.561861,0.468781,1.768965,0.815848,1.574333e+09,14,109,1.122607
176,-8.669962,20.681184,-1.282162,1.318819,0.345371,1.560515,0.624225,1.574333e+09,14,109,1.122607
177,3.419975,23.324231,-1.093944,6.501971,3.405857,1.808683,-1.367780,1.574333e+09,14,109,1.122607
178,6.497202,26.466779,-0.849282,2.031719,0.589699,1.267900,0.930146,1.574333e+09,14,109,1.122607
179,12.586931,-10.352997,-1.016283,8.512774,0.608562,1.663599,0.409952,1.574333e+09,14,130,1.167661
180,7.647049,-11.643031,-1.469142,1.621363,1.038405,0.780721,-0.160578,1.574333e+09,14,142,1.127531
181,-22.890851,-1.801824,-0.979447,1.887206,0.656564,2.248320,-1.369954,1.574333e+09,14,142,1.127531
182,-27.369268,5.134849,-0.901104,1.038909,0.543279,1.795716,-0.805184,1.574333e+09,14,142,1.127531
183,1.969348,20.032347,-0.729668,4.140657,0.735513,1.126964,0.372188,1.574333e+09,14,142,1.127531


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.461474,-13.737080,-0.769626),
  ID 7 - TimeNew 8
   - Last coords (2.437391,-32.143669,-0.591518),
  ID 6 - TimeNew 8
   - Last coords (7.631210,-26.516459,-1.283591),
  ID 4 - TimeNew 8
   - Last coords (11.461474,-13.737080,-0.769626),
  ID 3 - TimeNew 8
   - Last coords (6.497202,26.466779,-0.849282),
  ID 2 - TimeNew 8
   - Last coords (3.419975,23.324231,-1.093944),
  ID 1 - TimeNew 8
   - Last coords (-10.067497,17.093422,-1.345984),
  ID 0 - TimeNew 8
   - Last coords (-10.067497,17.093422,-1.345984),
  ID 9 - TimeNew 8
   - Last coords (9.886663,-6.064847,-1.084531),
  ID 12 - TimeNew 8
   - Last coords (-8.669962,20.681184,-1.282162),
  ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104),
  ID 17 - TimeNew 8
   - Last coords (12.586931,-10.352997,-1.016283),
  ID 21 - TimeNew 8
   - Last coords (-22.890851,-1.801824,-0.979447)],
 'Pending': [],
 'Inactive': [],
 'New': [ID 5 - TimeNew 15
   - Last coords (27.2096

In [80]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[5.49128956e-01 6.74144103e-03 6.35504529e-01 5.49128956e-01
 2.60089777e-02 4.32423015e-01 1.16356642e-01 1.16356642e-01
 3.25309126e-01 1.83334747e-01 8.60184163e+00 2.33424527e-01
 3.56309312e-01]
[10  8  9 10  4  2  1  1  0  3 -1  5  6]
Iou associations
[0. 0. 0. 0. 0.]
[-1 -1 -1 -1 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
187,9.860021,-5.826428,-0.864821,3.046099,0.827994,0.285660,1.497195,1.574333e+09,15,109,1.202583
188,-9.966296,17.118890,-1.397449,1.501983,0.710731,1.772616,0.710443,1.574333e+09,15,109,1.202583
189,3.504452,22.900140,-1.093946,6.527531,3.196607,1.807163,-1.373755,1.574333e+09,15,109,1.202583
190,-8.519338,20.761481,-1.215256,1.049007,0.449959,1.638179,0.786779,1.574333e+09,15,109,1.202583
191,6.499064,26.488451,-0.835023,2.066093,0.560873,1.256485,0.938008,1.574333e+09,15,109,1.202583
192,12.585630,-10.565862,-0.920503,9.052122,0.269675,2.015458,0.463406,1.574333e+09,15,130,1.247595
193,-22.780574,-2.140508,-0.970047,1.866210,0.711582,2.191321,-1.363940,1.574333e+09,15,142,1.207650
194,3.356968,20.364156,-0.541139,2.131343,0.907862,0.915034,0.494150,1.574333e+09,15,142,1.207650
195,2.440743,-32.149510,-0.591816,0.559959,0.468470,0.726834,-0.878038,1.574333e+09,15,143,1.247427
196,7.342640,-25.950256,-1.286287,6.400861,2.746377,2.704092,-0.774934,1.574333e+09,15,143,1.247427


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.502184,-14.284602,-0.779883),
  ID 7 - TimeNew 8
   - Last coords (2.440743,-32.149510,-0.591816),
  ID 6 - TimeNew 8
   - Last coords (7.342640,-25.950256,-1.286287),
  ID 4 - TimeNew 8
   - Last coords (11.502184,-14.284602,-0.779883),
  ID 3 - TimeNew 8
   - Last coords (6.499064,26.488451,-0.835023),
  ID 2 - TimeNew 8
   - Last coords (3.504452,22.900140,-1.093946),
  ID 1 - TimeNew 8
   - Last coords (-9.966296,17.118890,-1.397449),
  ID 0 - TimeNew 8
   - Last coords (-9.966296,17.118890,-1.397449),
  ID 9 - TimeNew 8
   - Last coords (9.860021,-5.826428,-0.864821),
  ID 12 - TimeNew 8
   - Last coords (-8.519338,20.761481,-1.215256),
  ID 17 - TimeNew 8
   - Last coords (12.585630,-10.565862,-0.920503),
  ID 21 - TimeNew 8
   - Last coords (-22.780574,-2.140508,-0.970047)],
 'Pending': [ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104)],
 'Inactive': [],
 'New': [ID 5 - TimeNew 16
   - Last coords (27.209659,9.8

In [81]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[7.30077821e-01 5.13380416e-03 6.51455178e-01 7.30077821e-01
 1.80439376e-02 3.91385452e-01 6.73635382e-02 6.73635382e-02
 9.33787144e+00 7.52240938e-02 4.57245111e+00 3.74978607e-01]
[ 8  6  7  8  3  1  0  0 -1  2 -1  4]
Iou associations
[0. 0. 0. 0.]
[-1 -1 -1 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
198,-9.986614,17.056164,-1.383650,1.828010,0.734084,1.889949,0.494564,1.574333e+09,16,109,1.279798
199,3.632373,22.530790,-1.113932,6.473916,3.203374,1.811958,-1.401757,1.574333e+09,16,109,1.279798
200,-8.590573,20.749124,-1.194483,1.047655,0.581912,1.774031,0.691099,1.574333e+09,16,109,1.279798
201,6.491386,26.503985,-0.829991,2.130560,0.567606,1.261172,0.922926,1.574333e+09,16,109,1.279798
202,-22.676631,-2.500391,-0.953045,1.938345,0.785215,2.123527,-1.287619,1.574333e+09,16,142,1.288589
203,1.903150,19.031716,-0.813420,5.272592,0.733321,0.787317,0.639061,1.574333e+09,16,142,1.288589
204,2.445814,-32.148985,-0.592416,0.591068,0.469995,0.730427,-0.807153,1.574333e+09,16,143,1.327392
205,7.034638,-25.376225,-1.282132,6.384089,2.709132,2.735497,-0.765504,1.574333e+09,16,143,1.327392
206,11.530424,-15.013515,-0.809897,7.692584,3.003993,1.529965,-1.559728,1.574333e+09,16,143,1.327392


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.530424,-15.013515,-0.809897),
  ID 7 - TimeNew 8
   - Last coords (2.445814,-32.148985,-0.592416),
  ID 6 - TimeNew 8
   - Last coords (7.034638,-25.376225,-1.282132),
  ID 4 - TimeNew 8
   - Last coords (11.530424,-15.013515,-0.809897),
  ID 3 - TimeNew 8
   - Last coords (6.491386,26.503985,-0.829991),
  ID 2 - TimeNew 8
   - Last coords (3.632373,22.530790,-1.113932),
  ID 1 - TimeNew 8
   - Last coords (-9.986614,17.056164,-1.383650),
  ID 0 - TimeNew 8
   - Last coords (-9.986614,17.056164,-1.383650),
  ID 12 - TimeNew 8
   - Last coords (-8.590573,20.749124,-1.194483),
  ID 21 - TimeNew 8
   - Last coords (-22.676631,-2.500391,-0.953045)],
 'Pending': [ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104),
  ID 17 - TimeNew 8
   - Last coords (12.585630,-10.565862,-0.920503),
  ID 9 - TimeNew 8
   - Last coords (9.860021,-5.826428,-0.864821)],
 'Inactive': [],
 'New': [ID 5 - TimeNew 17
   - Last coords (27.209659,9.8

In [82]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.47096567 0.00667823 0.69311057 0.47096567 0.02678458 0.45542092
 0.27123214 0.27123214 0.1005029  0.37374699]
[11  9 10 11  3  1  0  0  2  6]
Iou associations
[0. 0. 0. 0.]
[-1 -1 -1 -1]
KF associations
[30.28425346  6.77851596  9.30155236]
[-1 -1 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
207,-10.188196,16.882300,-1.331657,1.790226,0.668048,1.789975,0.315253,1.574333e+09,17,109,1.360119
208,3.623345,22.076763,-1.079491,6.387525,3.302216,1.826540,-1.355916,1.574333e+09,17,109,1.360119
209,-8.572111,20.760944,-1.292566,1.117079,0.514228,1.516860,0.822858,1.574333e+09,17,109,1.360119
210,6.493628,26.530497,-0.833080,2.184656,0.528331,1.260684,0.897548,1.574333e+09,17,109,1.360119
211,3.950849,-21.664217,-0.787448,2.934695,0.913325,1.146000,-0.530829,1.574333e+09,17,142,1.367596
212,7.257403,-14.755880,-0.965529,2.762661,1.290974,1.058589,-0.170225,1.574333e+09,17,142,1.367596
213,-22.537518,-2.846495,-0.929671,2.066068,0.766033,2.126305,-1.208694,1.574333e+09,17,142,1.367596
214,0.200259,17.666987,-0.952712,1.792857,0.273191,0.558455,0.546663,1.574333e+09,17,142,1.367596
215,4.126732,20.027884,-0.774141,2.961911,0.550529,0.234378,0.543958,1.574333e+09,17,142,1.367596
216,2.441758,-32.146087,-0.587971,0.594803,0.469645,0.732738,-0.726053,1.574333e+09,17,143,1.407398


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.651184,-15.468729,-0.812356),
  ID 7 - TimeNew 8
   - Last coords (2.441758,-32.146087,-0.587971),
  ID 6 - TimeNew 8
   - Last coords (6.709325,-24.764347,-1.295521),
  ID 4 - TimeNew 8
   - Last coords (11.651184,-15.468729,-0.812356),
  ID 3 - TimeNew 8
   - Last coords (6.493628,26.530497,-0.833080),
  ID 2 - TimeNew 8
   - Last coords (3.623345,22.076763,-1.079491),
  ID 1 - TimeNew 8
   - Last coords (-10.188196,16.882300,-1.331657),
  ID 0 - TimeNew 8
   - Last coords (-10.188196,16.882300,-1.331657),
  ID 12 - TimeNew 8
   - Last coords (-8.572111,20.760944,-1.292566),
  ID 21 - TimeNew 8
   - Last coords (-22.537518,-2.846495,-0.929671)],
 'Pending': [ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104),
  ID 17 - TimeNew 8
   - Last coords (12.585630,-10.565862,-0.920503),
  ID 9 - TimeNew 8
   - Last coords (9.860021,-5.826428,-0.864821)],
 'Inactive': [],
 'New': [ID 5 - TimeNew 18
   - Last coords (27.209659,9

In [83]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.6703904  0.00727519 0.67780212 0.6703904  0.02331445 0.40853323
 0.09484052 0.09484052 0.29004767 0.35300044]
[10  8  9 10  4  2  1  1  3  6]
Iou associations
[0.    0.    0.    0.    0.495 0.   ]
[-1 -1 -1 -1  1 -1]
KF associations
[38.98158442  4.71272218  0.99468645]
[-1 -1  0]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
219,9.736090,-6.813066,-0.840579,1.457154,0.596888,0.495198,1.328037,1.574333e+09,18,109,1.441141
220,-10.251379,16.814374,-1.311946,1.742257,0.675760,1.668646,0.159073,1.574333e+09,18,109,1.441141
221,3.730095,21.682423,-1.079952,6.339027,2.994577,1.834236,-1.298695,1.574333e+09,18,109,1.441141
222,-8.315247,20.895621,-1.289328,0.956164,0.598552,1.602073,0.738300,1.574333e+09,18,109,1.441141
223,6.501102,26.550952,-0.841404,2.222841,0.534946,1.250317,0.901460,1.574333e+09,18,109,1.441141
224,3.580220,-21.082588,-0.962836,3.381155,0.941028,1.342250,-0.714323,1.574333e+09,18,142,1.447652
225,-22.422524,-3.179359,-0.953906,1.942772,0.706939,2.123171,-1.225122,1.574333e+09,18,142,1.447652
226,0.118808,17.574942,-0.935689,2.104184,0.330795,0.757933,0.491741,1.574333e+09,18,142,1.447652
227,2.439046,-32.139782,-0.590384,0.537837,0.478072,0.738784,-0.488680,1.574333e+09,18,143,1.487554
228,6.383942,-24.169764,-1.292017,6.476807,2.756963,2.790097,-0.760323,1.574333e+09,18,143,1.487554


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.711833,-16.136232,-0.798752),
  ID 7 - TimeNew 8
   - Last coords (2.439046,-32.139782,-0.590384),
  ID 6 - TimeNew 8
   - Last coords (6.383942,-24.169764,-1.292017),
  ID 4 - TimeNew 8
   - Last coords (11.711833,-16.136232,-0.798752),
  ID 3 - TimeNew 8
   - Last coords (6.501102,26.550952,-0.841404),
  ID 2 - TimeNew 8
   - Last coords (3.730095,21.682423,-1.079952),
  ID 1 - TimeNew 8
   - Last coords (-10.251379,16.814374,-1.311946),
  ID 0 - TimeNew 8
   - Last coords (-10.251379,16.814374,-1.311946),
  ID 12 - TimeNew 8
   - Last coords (-8.315247,20.895621,-1.289328),
  ID 21 - TimeNew 8
   - Last coords (-22.422524,-3.179359,-0.953906),
  ID 9 - TimeNew 8
   - Last coords (9.736090,-6.813066,-0.840579)],
 'Pending': [ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104),
  ID 17 - TimeNew 8
   - Last coords (12.585630,-10.565862,-0.920503)],
 'Inactive': [],
 'New': [ID 5 - TimeNew 19
   - Last coords (27.209659,9

In [84]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[ 0.75982825  0.01761529  0.69489877  0.75982825  0.02823045  0.43195148
  0.07105468  0.07105468  0.05184662  0.37223203 10.28721135]
[11  9 10 11  3  1  0  0  2  6 -1]
Iou associations
[0.   0.   0.   0.   0.72]
[-1 -1 -1 -1  1]
KF associations
[31.21478035 27.84776888]
[-1 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
230,-10.184213,16.837174,-1.316150,2.202279,0.659036,1.646667,0.122524,1.574333e+09,19,109,1.520276
231,3.821918,21.260450,-1.070495,6.327831,2.891118,1.828549,-1.248904,1.574333e+09,19,109,1.520276
232,-8.322978,20.894379,-1.238076,0.660514,0.556395,1.549585,1.090681,1.574333e+09,19,109,1.520276
233,6.514946,26.575554,-0.841623,2.239806,0.505003,1.241557,0.903435,1.574333e+09,19,109,1.520276
234,5.192457,16.282450,-1.006061,3.009406,0.269296,0.725426,-0.292515,1.574333e+09,19,130,1.567736
235,3.391548,-21.121336,-0.927351,4.449912,1.779591,1.433663,-0.880867,1.574333e+09,19,142,1.527638
236,-22.282376,-3.524170,-0.949395,1.910625,0.734653,2.074624,-1.210440,1.574333e+09,19,142,1.527638
237,1.933285,15.879726,-0.381004,6.425747,0.626804,0.207947,0.454161,1.574333e+09,19,142,1.527638
238,0.670305,16.967505,-1.067574,3.889936,2.159806,1.148761,0.368338,1.574333e+09,19,142,1.527638
239,2.452844,-32.144191,-0.600408,0.625262,0.475284,0.750599,-0.804837,1.574333e+09,19,143,1.567590


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.801026,-16.890801,-0.795840),
  ID 7 - TimeNew 8
   - Last coords (2.452844,-32.144191,-0.600408),
  ID 6 - TimeNew 8
   - Last coords (6.056784,-23.556793,-1.302885),
  ID 4 - TimeNew 8
   - Last coords (11.801026,-16.890801,-0.795840),
  ID 3 - TimeNew 8
   - Last coords (6.514946,26.575554,-0.841623),
  ID 2 - TimeNew 8
   - Last coords (3.821918,21.260450,-1.070495),
  ID 1 - TimeNew 8
   - Last coords (-10.184213,16.837174,-1.316150),
  ID 0 - TimeNew 8
   - Last coords (-10.184213,16.837174,-1.316150),
  ID 12 - TimeNew 8
   - Last coords (-8.322978,20.894379,-1.238076),
  ID 21 - TimeNew 8
   - Last coords (-22.282376,-3.524170,-0.949395)],
 'Pending': [ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104),
  ID 17 - TimeNew 8
   - Last coords (12.585630,-10.565862,-0.920503),
  ID 9 - TimeNew 8
   - Last coords (9.736090,-6.813066,-0.840579)],
 'Inactive': [],
 'New': [ID 5 - TimeNew 20
   - Last coords (27.209659,9

In [85]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.73214809 0.02523253 0.68532415 0.73214809 0.01685419 0.46720926
 0.18973919 0.18973919 0.10031075 0.43587204]
[12 10 11 12  3  1  0  0  2  6]
Iou associations
[0.    0.    0.    0.    0.645 0.315]
[-1 -1 -1 -1  1  0]
KF associations
[29.3329379  30.01318863 25.6877208 ]
[-1 -1 -1]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
242,-10.309473,16.696137,-1.295665,1.997334,0.852307,1.603024,0.027652,1.574333e+09,20,109,1.599908
243,3.922451,20.804186,-1.071025,6.301021,2.665906,1.831764,-1.211268,1.574333e+09,20,109,1.599908
244,-8.315472,20.914971,-1.140189,1.245582,0.410121,1.443833,0.767632,1.574333e+09,20,109,1.599908
245,6.524989,26.588513,-0.837717,2.231642,0.493838,1.232713,0.924550,1.574333e+09,20,109,1.599908
246,5.724435,15.772092,-1.098952,1.952673,1.158571,0.769741,-0.379129,1.574333e+09,20,130,1.647728
247,3.069264,-21.051092,-1.011398,6.074778,2.502523,2.491393,-0.810920,1.574333e+09,20,142,1.607836
248,-22.103599,-3.920089,-0.985053,1.711914,0.828836,2.063385,-1.193181,1.574333e+09,20,142,1.607836
249,-0.533741,16.964352,-1.482997,2.383493,1.265987,1.615003,0.544680,1.574333e+09,20,142,1.607836
250,4.467917,18.328625,-0.867550,3.773783,0.851724,0.524996,0.504098,1.574333e+09,20,142,1.607836
251,2.079196,19.972037,-0.907856,2.990296,0.611075,2.371780,0.434410,1.574333e+09,20,142,1.607836


{'Active': [ID 8 - TimeNew 8
   - Last coords (11.914257,-17.613685,-0.821521),
  ID 7 - TimeNew 8
   - Last coords (2.438619,-32.126745,-0.589007),
  ID 6 - TimeNew 8
   - Last coords (5.711879,-22.964714,-1.315257),
  ID 4 - TimeNew 8
   - Last coords (11.914257,-17.613685,-0.821521),
  ID 3 - TimeNew 8
   - Last coords (6.524989,26.588513,-0.837717),
  ID 2 - TimeNew 8
   - Last coords (3.922451,20.804186,-1.071025),
  ID 1 - TimeNew 8
   - Last coords (-10.309473,16.696137,-1.295665),
  ID 0 - TimeNew 8
   - Last coords (-10.309473,16.696137,-1.295665),
  ID 12 - TimeNew 8
   - Last coords (-8.315472,20.914971,-1.140189),
  ID 21 - TimeNew 8
   - Last coords (-22.103599,-3.920089,-0.985053)],
 'Pending': [],
 'Inactive': [ID 9 - TimeNew 8
   - Last coords (9.736090,-6.813066,-0.840579),
  ID 17 - TimeNew 8
   - Last coords (12.585630,-10.565862,-0.920503),
  ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104)],
 'New': [ID 5 - TimeNew 21
   - Last coords (27.209659,9

In [86]:
TrackTrack.next_step()
display(TrackTrack.current_objects)
TrackTrack.tracks

KF associations
[0.57061339 0.02868454 0.63877006 0.57061339 0.02492757 0.46948321
 0.09035267 0.09035267 0.23346288 0.35572802]
[14 12 13 14  3  1  0  0  2  7]
Iou associations
[0.    0.    0.    0.    0.63  0.785 0.48 ]
[-1 -1 -1 -1  1  0  4]


,X_box,Y_box,Z_box,length_box,width_box,height_box,angle_box,timestamp,frame,camera,elapsed_time
255,-10.345101,16.614414,-1.310348,1.682396,1.111229,1.633540,-0.226735,1.574333e+09,21,109,1.683341
256,4.031595,20.347791,-1.056684,6.344770,2.523965,1.850589,-1.181636,1.574333e+09,21,109,1.683341
257,-8.099512,20.998620,-1.169665,0.735845,0.557080,1.408767,0.588384,1.574333e+09,21,109,1.683341
258,6.521189,26.567256,-0.825263,2.232152,0.562733,1.207123,0.951284,1.574333e+09,21,109,1.683341
259,5.604898,15.602146,-1.095399,1.936882,1.537682,0.831729,-0.105670,1.574333e+09,21,130,1.727656
260,3.373391,-21.211568,-1.098493,4.227082,3.228448,2.670570,-0.065334,1.574333e+09,21,142,1.687656
261,7.862506,-16.853498,-0.881723,1.216469,0.627697,0.692648,-0.535294,1.574333e+09,21,142,1.687656
262,-21.972788,-4.250887,-0.983132,1.760713,0.756044,2.014264,-1.172772,1.574333e+09,21,142,1.687656
263,4.390042,15.785769,-0.453855,1.708591,0.925691,0.350791,0.455752,1.574333e+09,21,142,1.687656
264,0.174555,17.251275,-1.281587,4.958470,0.656812,2.072166,0.697676,1.574333e+09,21,142,1.687656


{'Active': [ID 8 - TimeNew 8
   - Last coords (12.071443,-18.161988,-0.805548),
  ID 7 - TimeNew 8
   - Last coords (2.460345,-32.133113,-0.606621),
  ID 6 - TimeNew 8
   - Last coords (5.386537,-22.415187,-1.301152),
  ID 4 - TimeNew 8
   - Last coords (12.071443,-18.161988,-0.805548),
  ID 3 - TimeNew 8
   - Last coords (6.521189,26.567256,-0.825263),
  ID 2 - TimeNew 8
   - Last coords (4.031595,20.347791,-1.056684),
  ID 1 - TimeNew 8
   - Last coords (-10.345101,16.614414,-1.310348),
  ID 0 - TimeNew 8
   - Last coords (-10.345101,16.614414,-1.310348),
  ID 12 - TimeNew 8
   - Last coords (-8.099512,20.998620,-1.169665),
  ID 21 - TimeNew 8
   - Last coords (-21.972788,-4.250887,-0.983132)],
 'Pending': [],
 'Inactive': [ID 9 - TimeNew 8
   - Last coords (9.736090,-6.813066,-0.840579),
  ID 17 - TimeNew 8
   - Last coords (12.585630,-10.565862,-0.920503),
  ID 16 - TimeNew 8
   - Last coords (-27.369268,5.134849,-0.901104)],
 'New': [ID 5 - TimeNew 22
   - Last coords (27.209659,9

In [93]:
TrackTrack.tracks['Active'][7].objects

[x: -11.148409, y:15.363879, z:-0.941920, length:1.177285, width:0.561967, height:1.334849, angle:0.631927, camera:109 ,
 x: -11.137152, y:15.415920, z:-0.911968, length:1.352618, width:0.546684, height:1.370234, angle:0.708852, camera:109 ,
 x: -11.204201, y:15.463382, z:-0.987747, length:1.413320, width:0.406469, height:1.369057, angle:0.777954, camera:109 ,
 x: -11.043230, y:15.646841, z:-1.029184, length:1.613723, width:0.387612, height:1.470045, angle:0.722361, camera:109 ,
 x: -11.094020, y:15.663410, z:-0.992140, length:1.545281, width:0.490576, height:1.329137, angle:0.708912, camera:109 ,
 x: -11.163822, y:15.705071, z:-0.987490, length:1.702332, width:0.401186, height:1.166153, angle:0.686620, camera:109 ,
 x: -11.288490, y:15.691054, z:-1.057446, length:2.206232, width:0.351428, height:1.191374, angle:0.689989, camera:109 ,
 x: -11.014090, y:15.942268, z:-1.113405, length:2.132019, width:0.354482, height:1.242151, angle:0.701535, camera:109 ,
 x: -10.931569, y:16.055569, z:-